In [5]:
from IPython.display import display, Image, clear_output, HTML
import time
import random
import requests
from bs4 import BeautifulSoup
import json
import ipywidgets as widgets
from jupyter_ui_poll import ui_events

random.seed(1)

# Defining the image variables for the ANS test
# 4:3 images
test12_9 = Image("9_12.png", width = 600)
test16_12 = Image("16_12.png", width = 600)
test20_15 = Image("20_15.png", width = 600)
# 7:6 images
test14_12 = Image("14_12.png", width = 600)
test21_18 = Image("18_21.png", width = 600)
# 9:8 images
test18_16 = Image("18_16.png", width = 600)
# 10:9 images
test10_9 = Image("10_9.png", width = 600)
test20_18 = Image("18_20.png", width = 600)
blank_placeholder = Image("blank_placeholder.png", width = 600)

# List of all ANS images used for reshuffling later
ans_images_global = []
ans_images_global.append(test12_9)
ans_images_global.append(test16_12)
ans_images_global.append(test20_15)
ans_images_global.append(test14_12)
ans_images_global.append(test21_18)
ans_images_global.append(test18_16)
ans_images_global.append(test10_9)
ans_images_global.append(test20_18)

# Correct answers dictionary
ans_answers_global = {test12_9:'r', test16_12:'l', test20_15:'l', test14_12:'l', test21_18:'r', test18_16:'l',
                     test10_9:'l', test20_18:'r'}
# For converting the cycled test to a str for easy identification of question
ans_conversions_global = {test12_9:'9_12', test16_12:'16_12', test20_15:'20_15', test14_12:'14_12', 
                          test21_18:'18_21', test18_16:'18_16', test10_9:'10_9', test20_18:'18_20'}

# Defining buttons/widgets & dictionaries to hold results
event_info = {
    'type': '',
    'description': '',
    'time': -1
}

# Temp holds users answers for upload to google form
#user_id_dict = {'user_id': ''}
user_answers_dict = {
    'user_id': '',
    '9_12': '',
    '16_12': '',
    '20_15': '',
    '14_12': '',
    '18_21': '',
    '18_16': '',
    '10_9': '',
    '18_20': ''
}
form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSdZ8_rvGoTnt4F5J_2jYSXv1cyikRP6IUdHkIX1ziUH2pVAig/viewform?usp=sf_link'


# Wait for event function
def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info


# this function lets buttons 
# register events when clicked
def register_btn_event(btn):
    event_info['type'] = "button click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return


# Function to send user data to a google form
def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok


def single_ans_test(ans_images, ans_answers):
    # Defining variables and buttons
    form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSdZ8_rvGoTnt4F5J_2jYSXv1cyikRP6IUdHkIX1ziUH2pVAig/viewform?usp=sf_link'
    images_list = ans_images.copy()
    random.shuffle(images_list)
    score = 0
    btn1 = widgets.Button(description="Left")
    btn2 = widgets.Button(description="Right")
    btn1.on_click(register_btn_event)
    btn2.on_click(register_btn_event)
    panel = widgets.HBox([btn1, btn2])
    
    # Asking for User Agreement
    data_consent_info = """DATA CONSENT INFORMATION:
    
    Please read:
    
    we wish to record your response data
    
    to an anonymised public data repository.
    
    Your data will be used for educational teaching purposes
    
    practising data analysis and visualisation.
    
    Please type yes in the box below if you consent to the upload."""

    print(data_consent_info)
    result = input("> ")
    clear_output(wait=True)
    if result == "yes":
        print("Thanks for your participation.")
        print("Please contact philip.lewis@ucl.ac.uk")
        print("If you have any questions or concerns")
        print("regarding the stored results.")
    else:
        # end code execution by raising an exception
        raise(Exception("User did not consent to continue test."))
    time.sleep(5)
    clear_output(wait=True)
    
    # Creating a Unique User ID
    id_instructions = """

    Enter your anonymised ID
    
    To generate an anonymous 4-letter unique user identifier please enter:
    
    - two letters based on the initials (first and last name) of a childhood friend
    
    - two letters based on the initials (first and last name) of a favourite actor / actress
    
    e.g. if your friend was called Charlie Brown and film star was Tom Cruise
    
    then your unique identifer would be CBTC

    """

    print(id_instructions)
    
    user_answers_dict['user_id'] = input("> ")
    clear_output(wait=True)
    print("User entered id:", user_answers_dict['user_id'])
    
    time.sleep(4)
    clear_output(wait=True)
    
    print("""Welcome to the ANS test\n
    You will be shown two dot arrays with different numbers of dots.\n
    Please judge as quickly and accurately which of the arrays contains a greater number of dots.\n
    Please press the button left or right to select your answer.
    """)
    time.sleep(5)
    clear_output(wait=False)
    
    for i in images_list:
        # Variable recording which question/picture is being shown
        question_id = ans_conversions_global[i]
        
        # Showing ANS pictures
        clear_output(wait=True)
        display(i)
        time.sleep(0.75)
        clear_output(wait=True)
        display(blank_placeholder)      
        
        # Implementing buttons and waiting for input
        display(panel)
        result = wait_for_event(timeout = 3)
        clear_output(wait=True)
        display(blank_placeholder)
        
        # Recording user decision and confirming whether correct
        if result['description'] == 'Left':
            answer = 'l'
            if answer == ans_answers[i]:
                score += 1
                user_answers_dict[question_id] = True
            elif answer != ans_answers[i]:
                user_answers_dict[question_id] = False
                
        elif result['description'] == 'Right':
            answer = 'r'
            if answer == ans_answers[i]:
                score += 1
                user_answers_dict[question_id] = True
            elif answer != ans_answers[i]:
                user_answers_dict[question_id] = False
        elif result['description'] == '':
            print("No button was pressed! Answer more quickly")
            user_answers_dict[question_id] = 'NA'

        time.sleep(1.5)
    
    # Sending dictionary of user answers to google form
    #send_to_google_form(user_id_dict, form_url)
    send_to_google_form(user_answers_dict, form_url)
    clear_output(wait=False)
    return f"Your approximation for {score} of the {len(images_list)} image arrays were correct."

single_ans_test(ans_images_global, ans_answers_global)

'Your approximation for 7 of the 8 image arrays were correct.'